In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from processor.processor import Processor as p
from database.adatabase import ADatabase
from tabula import read_pdf
from fuzzywuzzy import fuzz

C:\Users\eric\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [84]:
income_statement_columns = {"net_revenue":["net_sales",""],
           "total_operating_expenses":[],
           "operating_income":[],
           "other_income":[],
           "income_before_provision_tax":[],
           "net_income":[]}
financial_statement_columns = {
           "cash_cash_equivalents":[],
           "account_receivables":[],
           "inventory":[],
           "current_assets":[],
           "property_plant_equipment":[],
           "intangible":[],
           "other_assets":[],
           "total_assets":[],
           "account_payables":[],
           "current_liabilities":[],
           "total_liabilities":[]}
columns = [income_statement_columns,financial_statement_columns]
annuals_db = ADatabase("annuals")

In [107]:
def find_statement(tables,columns):
    keywords = list(set("_".join(x for x in columns.keys()).split("_")))
    keyword_matches = {}
    max_match = 0
    for i in range(len(tables)):
        table = tables[i].copy()
        try:
            cleaned_table = table.rename(columns={table.columns[0]:"variables"}).copy()
            cleaned_table["variables"] = [str(x).lower().replace(" ","_") for x in cleaned_table["variables"]]
            all_vars = "_".join(cleaned_table["variables"]).split("_")
            overlaps = len([x for x in keywords if x in all_vars])
            keyword_matches[i] = overlaps
            max_match = max(overlaps,max_match)
            if max_match > len(keywords):
                break
        except Exception as e:
            print(str(e))
            continue
    for key in keyword_matches.keys():
        if keyword_matches[key] == max_match:
            return tables[key]
        else:
            continue

In [108]:
def value_cleaner(value):
    if "," in str(value) and "(" in str(value):
        return float(value[1:-1].replace(",",""))
    else:
        if "," in str(value):
            return float(value.replace(",",""))
        else:
            if "(" in str(value):
                return float(value[1:-1])
            else:
                if "—" in (str(value)):
                    return 0
                else:
                    return float(value)

In [109]:
year = 2005
base_url = "https://d1lge852tjjqow.cloudfront.net/CIK-0000320193/8857f118-4943-4198-8089-38d74a8b6646.pdf"
tables = read_pdf(base_url,pages="all",encoding="cp1252")

Got stderr: May 19, 2023 2:18:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 19, 2023 2:18:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 19, 2023 2:18:46 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 2:18:46 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 2:19:08 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 2:19:09 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [110]:
stuff_bruh = []
ticker = "AAPL"
incstate = find_statement(tables,columns[0])
financial_statement = find_statement(tables,columns[1])
statements = [incstate,financial_statement]
all_cols = []
for i in range(2):
    table = statements[i]
    cols = columns[i]
    example = table.copy()
    try:
        cleaned_table = example.rename(columns={example.columns[0]:"variables"}).copy()
        for col in list(example.columns[1:]):
            try:
                cleaned_table["values"] = cleaned_table[col].apply(value_cleaner)
                break
            except Exception as e:
                continue
        final = cleaned_table.pivot_table(values="values",columns="variables")
        included_cols = []
        for column in cols:
            column_ratios = []
            for col in final.columns:
                cleaned_col = col.lower().replace(" ","_")
                ratio = fuzz.ratio(column,cleaned_col)
                column_ratios.append({"column_name":col,"ratio":ratio})
            results = pd.DataFrame(column_ratios)
            final_col = results.sort_values("ratio",ascending=False).iloc[0]["column_name"]
            included_cols.append({"target":column,"attempt":final_col})
        for stuff in included_cols:
            final.rename(columns={stuff["attempt"]:stuff["target"]},inplace=True)
        stuff_bruh.append(final[[x for x in cols if x in final.columns]])
        all_cols.extend(included_cols)
    except Exception as e:
        continue
base = stuff_bruh[0]
base["year"] = year
for stuff in stuff_bruh[1:]:
    stuff["year"] = year
    base = base.merge(stuff,on="year",how="left")
base["ticker"] = ticker
base["year"] = year

IndexError: list index out of range

In [112]:
incstate

,"Segment operating income $ 1,661",Unnamed: 0,$ 989,Unnamed: 1,$ 620
0,Retail manufacturing margin (a) 435,NaN,213,NaN,106
1,"Corporate expenses, net (b) (446 )",NaN,(853 ),NaN,(701 )
2,Restructuring costs —,NaN,(23 ),NaN,(26 )
3,"Consolidated operating income (loss) $ 1,650",NaN,$ 326,NaN,$ (1 )
4,"Segment assets $ 1,881",NaN,"$ 1,411",NaN,"$ 1,197"
5,"Corporate assets 9,670",NaN,"6,639",NaN,"$ 5,618"
6,"Consolidated assets $ 11,551",NaN,"$ 8,050",NaN,"$ 6,815"
7,"Segment depreciation, amortization, and accret...",NaN,$ 49,NaN,$ 39
8,"Corporate depreciation, amortization, and accr...",NaN,101,NaN,74
9,"Consolidated depreciation, amortization, and a...",NaN,$ 150,NaN,$ 113


In [99]:
final = aapl_parse(tables,year,columns)
final

IndexError: list index out of range

In [ ]:
annuals_db.connect()
annuals_db.store("10k",final)
annuals_db.disconnect()